In [1]:
# pip install imblearn

In [44]:
import warnings
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

import sklearn.utils 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import imblearn
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler 
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold, LeaveOneOut

from sklearn.metrics import f1_score, recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
import numpy as np

from sklearn.tree import DecisionTreeClassifier



In [45]:
%ls

 Volume in drive C is Windows 
 Volume Serial Number is 04F5-FC96

 Directory of C:\Users\Yamlak\Documents\Summer Internship\Hamoye\Hamoye Projects\Stage C

26/06/2023  11:30    <DIR>          .
23/06/2023  09:30    <DIR>          ..
23/06/2023  09:32    <DIR>          .ipynb_checkpoints
26/06/2023  11:30            43,148 Ecological Footprint Anlaysis.ipynb
23/06/2023  09:33    <DIR>          footprint-nfa-2019-edition
               1 File(s)         43,148 bytes
               4 Dir(s)  170,513,563,648 bytes free


In [46]:
df = pd.read_csv( 'https://query.data.world/s/wh6j7rxy2hvrn4ml75ci62apk5hgae' ) 
df

,country,year,country_code,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,QScore
0,Armenia,1992,1,AreaPerCap,1.402924e-01,1.995463e-01,0.097188051,3.688847e-02,2.931995e-02,0.000000e+00,5.032351e-01,3A
1,Armenia,1992,1,AreaTotHA,4.830000e+05,6.870000e+05,334600,1.270000e+05,1.009430e+05,0.000000e+00,1.732543e+06,3A
2,Armenia,1992,1,BiocapPerCap,1.598044e-01,1.352610e-01,0.084003213,1.374213e-02,3.339780e-02,0.000000e+00,4.262086e-01,3A
3,Armenia,1992,1,BiocapTotGHA,5.501762e+05,4.656780e+05,289207.1078,4.731155e+04,1.149823e+05,0.000000e+00,1.467355e+06,3A
4,Armenia,1992,1,EFConsPerCap,3.875102e-01,1.894622e-01,1.26E-06,4.164833e-03,3.339780e-02,1.114093e+00,1.728629e+00,3A
...,...,...,...,...,...,...,...,...,...,...,...,...
72181,World,2016,5001,BiocapTotGHA,3.984702e+09,1.504757e+09,5111762779.0,1.095445e+09,4.726163e+08,0.000000e+00,1.216928e+10,3A
72182,World,2016,5001,EFConsPerCap,5.336445e-01,1.402092e-01,0.273495,8.974253e-02,6.329435e-02,1.646235e+00,2.746619e+00,3A
72183,World,2016,5001,EFConsTotGHA,3.984702e+09,1.046937e+09,2042179333.0,6.701039e+08,4.726163e+08,1.229237e+10,2.050891e+10,3A
72184,World,2016,5001,EFProdPerCap,5.336445e-01,1.402092e-01,0.273495,8.974253e-02,6.329435e-02,1.646235e+00,2.746619e+00,3A


In [47]:
df[ 'QScore' ].value_counts()

3A    51481
2A    10576
2B    10096
1A       16
1B       16
Name: QScore, dtype: int64

In [48]:
df.isna().sum() 


country               0
year                  0
country_code          0
record                0
crop_land         20472
grazing_land      20472
forest_land       20472
fishing_ground    20473
built_up_land     20473
carbon            20473
total                 9
QScore                1
dtype: int64

In [49]:
df = df.dropna()
df.isna().sum() 


country           0
year              0
country_code      0
record            0
crop_land         0
grazing_land      0
forest_land       0
fishing_ground    0
built_up_land     0
carbon            0
total             0
QScore            0
dtype: int64

In [50]:
df[ 'QScore' ] = df[ 'QScore' ].replace([ '1A' ], '2A' )
df.QScore.value_counts() 

3A    51473
2A      240
Name: QScore, dtype: int64

In [51]:
df_2A = df[df.QScore== '2A' ]
df_3A = df[df.QScore== '3A' ].sample( 350 )
data_df = df_2A.append(df_3A) 
data_df

,country,year,country_code,record,crop_land,grazing_land,forest_land,fishing_ground,built_up_land,carbon,total,QScore
1536,Algeria,2016,4,AreaPerCap,2.072989e-01,8.112722e-01,0.048357265,2.258528e-02,2.998367e-02,0.000000,1.119497e+00,2A
1537,Algeria,2016,4,AreaTotHA,8.417600e+06,3.294260e+07,1963600,9.171000e+05,1.217520e+06,0.000000,4.545842e+07,2A
1538,Algeria,2016,4,BiocapPerCap,2.021916e-01,2.636077e-01,0.027166736,7.947991e-03,2.924496e-02,0.000000,5.301590e-01,2A
1539,Algeria,2016,4,BiocapTotGHA,8.210214e+06,1.070408e+07,1103135.245,3.227369e+05,1.187524e+06,0.000000,2.152769e+07,2A
1540,Algeria,2016,4,EFConsPerCap,6.280528e-01,1.810332e-01,0.162800822,1.472910e-02,2.924496e-02,1.391455,2.407316e+00,2A
...,...,...,...,...,...,...,...,...,...,...,...,...
56631,Somalia,1984,201,EFConsTotGHA,1.184389e+06,5.775389e+06,2267922.949,2.925310e+04,2.985595e+05,416523.235300,9.972036e+06,3A
64070,United Arab Emirates,2009,225,EFConsPerCap,9.940415e-01,3.940446e-01,0.3300169,2.581508e-01,0.000000e+00,9.089021,1.106528e+01,3A
71443,China,1980,351,AreaTotHA,1.002190e+08,3.283000e+08,168005589.0,1.352290e+08,2.251980e+07,0.000000,7.542734e+08,3A
18699,El Salvador,2001,60,BiocapTotGHA,1.657349e+06,8.942484e+05,323222.7963,7.144222e+05,2.716779e+05,0.000000,3.860920e+06,3A


In [52]:
data_df = sklearn.utils.shuffle(data_df)
data_df = data_df.reset_index(drop= True )
data_df.shape
data_df.QScore.value_counts() 

3A    350
2A    240
Name: QScore, dtype: int64

In [53]:
data_df = data_df.drop(columns=[ 'country_code' , 'country' , 'year' ]) 

In [54]:
X = data_df.drop(columns= 'QScore' )
y = data_df[ 'QScore' ] 

In [55]:
#split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size= 0.3 , random_state= 0 )
y_train.value_counts() 


3A    251
2A    162
Name: QScore, dtype: int64

In [56]:
#encode categorical variable 
encoder = LabelEncoder()
x_train.record = encoder.fit_transform(x_train.record)
x_test.record = encoder.transform(x_test.record) 

In [15]:
smote = SMOTE(random_state= 1 )
x_train_balanced, y_balanced = smote.fit_resample(x_train, y_train) 


In [16]:
scaler = MinMaxScaler()
normalised_train_df = scaler.fit_transform(x_train_balanced.drop(columns=['record']))
normalised_train_df = pd.DataFrame(normalised_train_df,
columns=x_train_balanced.drop(columns=[ 'record' ]).columns)
normalised_train_df[ 'record' ] = x_train_balanced[ 'record' ] 


In [17]:
x_test = x_test.reset_index(drop= True )
print(x_test.shape)
normalised_test_df = scaler.transform(x_test.drop(columns=[ 'record' ]))
print(normalised_test_df.shape)
normalised_test_df = pd.DataFrame(normalised_test_df,columns=x_test.drop(columns=[ 'record' ]).columns)
print(normalised_test_df.shape)
normalised_test_df[ 'record' ] = x_test[ 'record' ] 
print(normalised_test_df.shape)


(177, 8)
(177, 7)
(177, 7)
(177, 8)


In [18]:
x_test.shape

(177, 8)

In [19]:
# Logistic Regression

log_reg = LogisticRegression()
log_reg.fit(normalised_train_df, y_balanced) 


LogisticRegression()

In [20]:
# K-Fold Cross validation

In [21]:
scores =  cross_val_score(log_reg, normalised_train_df, y_balanced, cv =5, scoring = 'f1_macro')
scores

array([0.42527174, 0.4265252 , 0.46420429, 0.4678442 , 0.52122241])

In [22]:
kf = KFold(n_splits =5)
kf.split(normalised_train_df)
f1_scores = []

for train_index, test_index in kf.split(normalised_train_df):
    x_Train, x_Test = normalised_train_df.iloc[train_index], normalised_train_df.iloc[test_index]
    y_Train, y_Test = y_balanced[train_index], y_balanced[test_index]
    model = LogisticRegression().fit(x_Train, y_Train)
    
    f1_scores.append(f1_score(y_true=y_Test, y_pred =model.predict(x_Test), pos_label = '2A')*100)

In [23]:
# Stratified KFold

In [24]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
f1_scores = []

for train_index, test_index in skf.split(normalised_train_df, y_balanced):
    x_Train, x_Test = np.array(normalised_train_df)[train_index], np.array(normalised_train_df)[test_index]
    y_Train, y_Test = y_balanced[train_index], y_balanced[test_index]
    model = LogisticRegression().fit(x_Train, y_Train)
    
    f1_scores.append(f1_score(y_true=y_Test, y_pred=model.predict(x_Test), pos_label='2A'))

In [25]:
# Leave One Out Cross Validation (LOOCV)

In [26]:
loo = LeaveOneOut()
scores = cross_val_score(LogisticRegression(), normalised_train_df, y_balanced, cv=loo, scoring='f1_macro')
average_score = scores.mean()*100
average_score

44.8936170212766

In [27]:
# Accuracy

In [28]:
print("Shape of y_test:", y_test.shape)
print("Shape of x_test:", x_test.shape)
# print("Shape of new_predictions:", new_predictions.shape)
normalised_test_df.shape

Shape of y_test: (177,)
Shape of x_test: (177, 8)


(177, 8)

In [29]:
new_predictions = log_reg.predict(normalised_test_df)
cnf_mat = confusion_matrix(y_true=y_test, y_pred=new_predictions, labels=[ '2A' , '3A' ])
cnf_mat 

array([[41, 21],
       [70, 45]], dtype=int64)

In [30]:
accuracy = accuracy_score(y_true=y_test, y_pred=new_predictions)
accuracy

0.4858757062146893

In [31]:
precision = precision_score(y_true = y_test, y_pred = new_predictions, pos_label='2A')
precision

0.36936936936936937

In [33]:
recall = recall_score(y_true=y_test, y_pred = new_predictions, pos_label='2A')
recall

0.6612903225806451

In [34]:
f1 = f1_score(y_true=y_test, y_pred= new_predictions, pos_label='2A')
f1

0.4739884393063583

In [36]:
# Decision Tree

In [37]:
dec_tree = DecisionTreeClassifier()
dec_tree.fit(normalised_train_df, y_balanced)

DecisionTreeClassifier()

In [ ]:
# Question 2

In [38]:
TP = 255
TN = 20
FP = 1380
FN =45

In [40]:
Accuracy = (TP + TN) / (TP + TN + FP + FN)
Recall = TP / (TP + FN)
Precision = TP / (TP + FP)

F1_Score = 2 * (Precision * Recall) / (Precision + Recall)
F1_Score

0.2635658914728682

In [ ]:
# Question 3

In [57]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Assuming you have already trained and obtained the random forest classifier 'rf' and the test set 'X_test'
# Make predictions on the test set
rf = RandomForestClassifier()
rf.fit(x_train, y_train) 

# Make predictions on the test set
y_pred = rf.predict(x_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy on the test set: {:.4f}".format(accuracy))

Accuracy on the test set: 0.7288


In [60]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import RandomizedSearchCV

param_grid = {
    'n_estimators': [100, 500, 300, 1000],
    'min_samples_split': [7, 2, 5, 2],
    'min_samples_leaf': [4, 8, 6, 8],
    'max_features': [None, 'log2', 'auto', None]
}


estimator = ExtraTreesClassifier(random_state=1)

random_search = RandomizedSearchCV(estimator=estimator, param_distributions=param_grid,
                                   scoring='accuracy', cv=5, n_iter=10,
                                   n_jobs=-1, verbose=1, random_state=1)

random_search.fit(x_train, y_train)

print("Best Hyperparameters:", random_search.best_params_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best Hyperparameters: {'n_estimators': 100, 'min_samples_split': 5, 'min_samples_leaf': 8, 'max_features': None}


In [62]:
selected_hyperparameters = {
    'n_estimators': 100,
    'min_samples_split': 5,
    'min_samples_leaf': 8,
    'max_features': None
}


model = ExtraTreesClassifier(random_state=1, **selected_hyperparameters)

# Fit the model on the training data
model.fit(x_train, y_train)

# Make predictions on the test set
y_pred = model.predict(x_test)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.6384180790960452

In [68]:
importances = model.feature_importances_
most_important_index = np.argmax(importances)
least_important_index = np.argmin(importances)



In [69]:
x_test.columns[most_important_index]

'crop_land'

In [70]:
x_test.columns[least_important_index]

'grazing_land'

In [71]:
x_test.columns

Index(['record', 'crop_land', 'grazing_land', 'forest_land', 'fishing_ground',
       'built_up_land', 'carbon', 'total'],
      dtype='object')